# Well Placement and Control Optimization using a spatiotemporal proxy
### Misael M. Morales - 2024
***

In [ ]:
import os
from tqdm import tqdm
from multiprocessing import Pool

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

from scipy.io import savemat, loadmat
from skimage.transform import resize
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
from torch import einsum
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split

from neuralop.models import *
from transformers import Swinv2Config, Swinv2Model

from utils import check_torch
device = check_torch()

sec2year   = 365.25 * 24 * 60 * 60
psi2pascal = 6894.76
co2_rho    = 686.5266
mega       = 1e6

n_timesteps = 33
nx, ny, nz  = 100, 100, 11

indexMap = loadmat('data_100_100_11/G_cells_indexMap.mat', simplify_cells=True)['gci']
Grid = np.zeros((nx,ny,nz)).flatten(order='F')
Grid[indexMap] = 1
Grid = Grid.reshape(nx,ny,nz, order='F')
Tops = np.load('data_npy_100_100_11/tops_grid.npz')['tops']

In [ ]:
history = pd.read_csv('MiONet_losses.csv')

def plot_loss(history, figsize=(8,4)):
    plt.figure(figsize=figsize)
    plt.plot(history.index, history['train'], ls='-', label='Train')
    plt.plot(history.index, history['valid'], ls='-', label='Valid')
    plt.xlabel('Epoch'); plt.ylabel('Loss')
    plt.legend(); plt.grid(True, which='both')
    plt.tight_layout(); plt.savefig('loss.png'); plt.show()
    return None
plot_loss(history)

***
# END